 ## Глава 1: Внимание к большим языковым моделям



 ### Содержание главы



 В этой главе мы:



 *   Кратко рассмотрим историю развития языковых моделей.

 *   Разберем основные элементы архитектуры Transformer и механизма внимания.

 *   Поймем различия между типами тонкой настройки.

 ### Архитектура Transformer



 ![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch1/stacked_layers.png?raw=True)

 <center>Рисунок 1.1 - "Слои" Transformer, объединенные в стек</center>



 ![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch1/full_transformer.png?raw=True)

 <center>Рисунок 1.2 - Детализированная архитектура Transformer</center>



 ![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch1/bert_embeddings.png?raw=True)

 <center>Рисунок 1.3 - Контекстные векторные представления слов из модели BERT</center>

 ### Внимание — это все, что нужно



 $$

 \Large

 \text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V

 $$

 <center>Уравнение 1.1 - Формула механизма внимания</center>



 ![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch1/translation_att.png?raw=True)

 <center>Рисунок 1.4 - Оценки внимания</center>



 ![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch1/multiple_keys_context.png?raw=True)

 <center>Рисунок 1.5 - Запрос к двумерным ключам</center>



 $$

 \Large

 \text{cos}\theta = ||Q|| ||K|| = Q \cdot K

 $$

 <center>Уравнение 1.2 - Косинусное сходство, нормы и скалярное произведение</center>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from huggingface_hub import snapshot_download

# Загружаем все файлы в локальную директорию
snapshot_download(
    repo_id="microsoft/Phi-3-mini-4k-instruct",
    local_dir="./phi3-mini-tokenizer",
    allow_patterns=[
        "tokenizer.json",
        "tokenizer_config.json",
        "special_tokens_map.json",
        "vocab.json",
        "merges.txt",
        "added_tokens.json"
    ]
)

# Затем загружаем из локальной директории
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("./phi3-mini-tokenizer")
vocab_size = len(tokenizer)

torch.manual_seed(13)
# Создадим искусственные слои векторных представлений и проекций
d_model = 1024
embedding_layer = nn.Embedding(vocab_size, d_model)
linear_query = nn.Linear(d_model, d_model)
linear_key = nn.Linear(d_model, d_model)
linear_value = nn.Linear(d_model, d_model)


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
sentence = 'Just a dummy sentence'
input_ids = tokenizer(sentence, return_tensors='pt')['input_ids']
input_ids


tensor([[ 3387,   263, 20254, 10541]])

In [3]:
embeddings = embedding_layer(input_ids)
embeddings.shape


torch.Size([1, 4, 1024])

In [4]:
# Проекции
proj_key = linear_key(embeddings)
proj_value = linear_value(embeddings)
proj_query = linear_query(embeddings)
# Оценки внимания
dot_products = torch.matmul(proj_query, proj_key.transpose(-2, -1))
scores = F.softmax(dot_products / np.sqrt(d_model), dim=-1)
scores.shape


torch.Size([1, 4, 4])

In [5]:
context = torch.matmul(scores, proj_value)
context.shape

torch.Size([1, 4, 1024])